In [38]:
import requests
from io import BytesIO
import pandas as pd
from zipfile import ZipFile
import re
from dtype_diet import report_on_dataframe, optimize_dtypes
import os

import warnings
warnings.filterwarnings('ignore')

In [6]:
path = 'https://portal.inmet.gov.br/uploads/dadoshistoricos/2022.zip'
r = requests.get(path, verify = False)
files = ZipFile(BytesIO(r.content))
pattern = re.compile("Porto Alegre", flags = re.IGNORECASE)
arquivo = []
for file in files.namelist():
    if re.search(pattern, str(file)):
        arquivo.append(file)
temp_poa = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", skiprows=8)

temp_poa_desc = pd.read_csv(files.open(arquivo[0]), encoding="latin1", sep = ";", nrows=7, header = None)
desc = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}

In [24]:
temp_poa.head()

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2022/01/01,0000 UTC,0,"1000,3","1000,3","999,4",NaN,25,"19,7","26,4",25,"19,7","19,3",72.0,65.0,72.0,126.0,"5,4","2,1",NaN
1,2022/01/01,0100 UTC,0,"1000,6","1000,6","1000,2",NaN,"24,1","19,8",25,"24,1","19,8","19,6",77.0,72.0,77.0,95.0,"4,3","1,3",NaN
2,2022/01/01,0200 UTC,0,"1000,5","1000,8","1000,5",NaN,"23,6","19,9","24,1","23,6","19,9","19,8",80.0,77.0,80.0,76.0,"3,4",1,NaN
3,2022/01/01,0300 UTC,0,1000,"1000,5",1000,NaN,"23,3","19,6","23,8","23,3","19,9","19,5",80.0,78.0,79.0,104.0,"2,6",1,NaN
4,2022/01/01,0400 UTC,0,"999,7",1000,"999,6",NaN,23,"20,1","23,6","22,5","20,2","19,4",85.0,78.0,84.0,132.0,"3,4","1,4",NaN


In [23]:
temp_poa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Data                                                   8760 non-null   object 
 1   Hora UTC                                               8760 non-null   object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8730 non-null   object 
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8730 non-null   object 
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8729 non-null   object 
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8729 non-null   object 
 6   RADIACAO GLOBAL (Kj/m²)                                4702 non-null   object 
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8730 non-null   object 
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)            

In [25]:
proposed_df = report_on_dataframe(temp_poa, unit="MB")
new_df = optimize_dtypes(temp_poa, proposed_df)
print(f'Original df memory: {temp_poa.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 7.92751407623291 MB
Propsed df memory: 0.9679117202758789 MB


In [26]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 20 columns):
 #   Column                                                 Non-Null Count  Dtype   
---  ------                                                 --------------  -----   
 0   Data                                                   8760 non-null   category
 1   Hora UTC                                               8760 non-null   category
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       8730 non-null   category
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  8730 non-null   category
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        8729 non-null   category
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       8729 non-null   category
 6   RADIACAO GLOBAL (Kj/m²)                                4702 non-null   category
 7   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           8730 non-null   category
 8   TEMPERATURA DO PONTO DE ORVALHO (°C)  

In [27]:
desc

{'REGIAO:': 'S',
 'UF:': 'RS',
 'ESTACAO:': 'PORTO ALEGRE - JARDIM BOTANICO',
 'CODIGO (WMO):': 'A801',
 'LATITUDE:': '-30,05361111',
 'LONGITUDE:': '-51,17472221',
 'ALTITUDE:': '41,18'}

In [53]:
df = pd.DataFrame()
for ano in range(2000, 2024):
    path = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{ano}.zip'
    r = requests.get(path, verify = False)
    files = ZipFile(BytesIO(r.content))
    arquivos = []
    for file in files.namelist():
        if file[-4:].lower() == ".csv":
            arquivos.append(file)
        else:
            continue
    df01 = pd.DataFrame()
    for arquivo in arquivos:
        info = pd.read_csv(files.open(arquivo), sep = ";", encoding = "latin-1", nrows=7, header = None)
        desc = {line[1][0]: line[1][1] for line in temp_poa_desc.iterrows()}
        df02 = pd.read_csv(files.open(arquivo),  sep = ";", encoding = "latin-1", skiprows = 8)
        df02["ESTACAO"] = desc['ESTACAO:']
        df02["UF"] = desc['UF:']
        df02["REGIAO"] = desc['REGIAO:']
        #nome_arquivo = arquivo.split('/')[1]
        #df02.to_parquet(f'../../../data/inmet/inmet_{nome_arquivo[:-4]}.parquet')
        df01 = pd.concat([df01, df02])
    df = pd.concat([df, df01])

IndexError: list index out of range